#Task 1: Import Libraries

In [2]:
!pip install transformers torch accelerate

In [13]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#Task 2: Load Lightweight model


In [14]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model Loaded Successfully ✅")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Model Loaded Successfully ✅


#Task 3: Initialise FAQ knowledge Base

In [15]:
faq_dict = {
    "return policy": "Our return policy allows returns within 30 days with a valid receipt.",
    "refund": "Refunds are processed within 5-7 business days after approval.",
    "shipping": "Standard shipping takes 3-5 business days.",
    "cancel order": "To cancel your order, please provide your order ID.",
    "contact": "You can contact our support team at support@company.com."
}


#Task 4: FAQ Detection Function


In [16]:
def check_faq(user_input):
    user_input = user_input.lower()
    for key in faq_dict:
        if key in user_input:
            return faq_dict[key]
    return None

In [17]:
#Conversation memory
conversation = [
    {"role": "system", "content": "You are a professional and polite customer service assistant. Always reply clearly and briefly."}
]

#Task 5: AI Response Generator(With Memory)

In [18]:
def generate_ai_response(user_input):

    global conversation

    conversation.append({"role": "user", "content": user_input})

    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.7,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

    reply = reply.split("assistant")[-1].strip()

    conversation.append({"role": "assistant", "content": reply})

    return reply

#Task 6: Initialise Chatbot Conversation

In [19]:
print("======================================")
print("🤖 Welcome to Smart Customer Service Chatbot")
print("Type 'exit' to end conversation")
print("======================================\n")

🤖 Welcome to Smart Customer Service Chatbot
Type 'exit' to end conversation



#Task 7: Testing section



In [20]:
test_questions = [
    "What is your return policy?",
    "I want a refund",
    "My order is delayed"
]

print("Running Test Cases...\n")

for question in test_questions:
    print("You:", question)

    faq_response = check_faq(question)

    if faq_response:
        print("Chatbot (FAQ):", faq_response)
    else:
        print("Chatbot (AI):", generate_ai_response(question))

    print("-" * 50)

Running Test Cases...

You: What is your return policy?
Chatbot (FAQ): Our return policy allows returns within 30 days with a valid receipt.
--------------------------------------------------
You: I want a refund
Chatbot (FAQ): Refunds are processed within 5-7 business days after approval.
--------------------------------------------------
You: My order is delayed
Chatbot (AI): |>
I'm sorry to hear that your order has been delayed. To assist you with this issue, please provide me with your order ID or order number. Once you have provided me with this information, I can investigate and let you know the current status of your order. Thank you for your understanding and patience. If you need further assistance, please do not hesitate to reach out to our customer
--------------------------------------------------


#Interactive Chat

In [21]:
while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("Chatbot: Thank you for contacting us. Have a great day! 😊")
        break

    faq_response = check_faq(user_input)

    if faq_response:
        print("Chatbot (FAQ):", faq_response)
    else:
        reply = generate_ai_response(user_input)
        print("Chatbot (AI):", reply)

You: hey!
Chatbot (AI): |>
|>
Thank you for reaching out to us. I'd be happy to assist you with your inquiry. Please provide me with your order ID or order number so I can investigate your issue further. I'll contact you shortly to let you know the status of your order. Have a great day! :D 😊
You: 20
Chatbot (AI): |>
|>
|>
Sure, here's the order ID: 12345678. And please let me know if you have any questions or concerns about your order. Best regards,

[Your Name]
[Your Company/Business Name]
You: Akshita
Chatbot (AI): |>
|>
Sure, I'd be happy to assist you. Please provide me with your full name and order ID so I can check your account and see if there's a delay with your order. Additionally, if you have any specific questions about the status of your order, please let me know. Best regards,

[Your Name]
[Your Company/Business Name
You: Clothes  35
Chatbot (AI): |>
|>
|>
|>
Sure, I'd be happy to assist you. Please provide me with your full name, email address, and order ID for your clot